In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle

from feature_engine.imputation import MeanMedianImputer, ArbitraryNumberImputer
from feature_engine.outliers import Winsorizer
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.discretisation import EqualFrequencyDiscretiser
from feature_engine.discretisation.arbitrary import ArbitraryDiscretiser
from feature_engine.selection import DropFeatures
from feature_engine.encoding import MeanEncoder

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler , MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn.pipeline import Pipeline 

s_path = os.path.dirname(os.path.realpath(__file__))
sys.path.append(s_path)

pd.set_option('display.max_columns',100)
pd.set_option('precision', 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import tm_teoriaMvto_base_prep as base_prep
import tm_teoriaMvto_label as tm_label
import tm_teoriaMvto_train as tm_train
import tm_teoriaMvto_ft_eng as ft_eng
# import tm_teoriaMvto_ft_sel as ft_sel

# models to test
from sklearn.svm import SVC
import xgboost as xgb


In C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Desktop\Anaconda3\envs\n

In [2]:
path_files = os.path.join(s_path, 'historical_files')

# base prep control
EXPORT_X = False
EXPORT_Y = False

# params labeling
s_prefix = 'ft_'
s_lbl_type = 'c_binary'
f_th = 0.2
b_dist = True
b_percent = True
b_custom = True

# params loading
s_regime = 'mi_up'



In [3]:
if EXPORT_X:
  df_data = base_prep.import_sampling(path_files= os.path.join(path_files, 'FeaturesFiles'), prefix_files= s_prefix)

  # create new features before extracting samples - using full data needed by lag features
  df_data = ft_eng.BasicFeatures().transform(X=df_data)

  df_X = base_prep.ft_export(df_data, path_files= os.path.join(path_files, 'TrainFiles'), prefix= s_prefix)
  y_sc = base_prep.y_export(df_data, path_files= os.path.join(path_files, 'TrainFiles'), prefix= s_prefix)



In [4]:
y_config = 'prefix_'+ s_prefix +'_type_'+ s_lbl_type +'_th_'+ str(f_th) +'_dist_'+ str(b_dist) +'_percent_'+str(b_percent)+'_custom_'+str(b_custom)

if EXPORT_Y:
  df_label = None 
  if EXPORT_X: df_label = y_sc

  # specify df_data = None (default) to load pickle from s_path + 'y_prep_data.pkl'
  label = tm_label.Labeling(df_data = df_label, label_type= s_lbl_type, b_dist_to_high= b_dist, 
                            s_path = os.path.join(path_files, 'TrainFiles'), prefix_files = s_prefix, th_label_y1=f_th, 
                            b_percent_freq=b_percent, b_custom_dir=b_custom)

  df_y = label.apply_label(s_model_return = s_regime)  # it will export 4 pickle files to label_obj.s_path
  # y_config = label.s_name

y_outfile = os.path.join(os.path.join(path_files, 'TrainFiles'), 'y_' + y_config +'.pkl')

if os.path.exists(y_outfile):
  print('y outfile ready for configuration: {}'.format(y_config))
else:
  print('WARNING: y outfile not found for configuration. Define EXPORT_Y = True and try again {}'.format(y_config))



y outfile ready for configuration: prefix_ft__type_c_binary_th_0.2_dist_True_percent_True_custom_True


In [5]:
l_prop_08 = ['escora_bid_2.5_0.8', 'escora_ask_2.5_0.8', 'escora_bid_3.5_0.8', 
            'escora_ask_3.5_0.8', 'escora_bid_4.5_0.8', 'escora_ask_4.5_0.8', ]

l_prop_12 = ['escora_bid_2.5_1.2', 'escora_ask_2.5_1.2', 'escora_bid_3.5_1.2',
            'escora_ask_3.5_1.2', 'escora_ask_4.5_1.2', 'escora_bid_4.5_1.2', ]

l_prop_2 = [ 'escora_bid_2.5_2', 'escora_ask_2.5_2', 'escora_bid_3.5_2',
            'escora_ask_3.5_2', 'escora_bid_4.5_2', 'escora_ask_4.5_2', ]

l_col_log = ['agg_net_d', 'aggbig_net_d', 'vol_trd', 'vol_big', 'big_v', 'vol_trd_aux', 
   'vol_big_aux', 'big_v_aux', 'loc_agg_net_d', 'big_c', 'big_c_aux',
   'loc_aggbig_net_d', 'agg_net_m', 'agg_net_m_aux', 'abagg', 'abagg_aux',
   'aggbig_net_m', 'aggbig_net_m_aux', 'loc_agg_net_m', 'loc_aggbig_net_m',
   'loc_agg_net_m_aux', 'loc_aggbig_net_m_aux', 'loc_aggbig_c_m', 'loc_aggbig_v_m', 
   'loc_aggbig_c_m_aux', 'loc_aggbig_v_m_aux', 'abs_v', 'abs_c', 'aggpior_v', 'aggpior_v_aux', 
   'aggpior_c', 'aggpior_c_aux', 'agg_net_10', 'agg_net_40', 'agg_net_80', 'loc_agg_net_10',
   'aggbig_net_10', 'aggpior_DIF', 'aggpior_DIF_30', 'abs_DIF', 'abs_DIF_30',
   'abagg_10', 'aggpior_aux_DIF', ]

l_side_drop = ['big_c','big_v','aggpior_c','aggpior_v','loc_aggbig_c_m','loc_aggbig_v_m','pagg_c_best',
  'pagg_c_best_0.5','pagg_c_best_0.7','pagg_c_best_0.9','pagg_v_best','pagg_v_best_0.5','pagg_v_best_0.7',
  'pagg_v_best_0.9','abs_c','abs_v','int_c','int_c_0.6','int_c_0.7','int_c_0.8','int_c_0.9','int_dif_c',
  'int_v','int_v_0.6','int_v_0.7','int_v_0.8','int_v_0.9','int_dif_v','imp_c','imp_c_0.6','imp_c_0.7',
  'imp_c_0.8','imp_c_0.9','imp_v','imp_v_0.6','imp_v_0.7','imp_v_0.8','imp_v_0.9','escora_bid_2.5_1.2',
  'escora_bid_2.5_2','escora_ask_2.5_1.2','escora_ask_2.5_2','escora_bid_3.5_1.2','escora_bid_3.5_2',
  'escora_ask_3.5_1.2','escora_ask_3.5_2','escora_bid_4.5_0.8','escora_bid_4.5_2','escora_ask_4.5_0.8',
  'escora_ask_4.5_2','movesc_bid_2.5','movesc_ask_2.5','movesc_bid_2.5_0.5','movesc_ask_2.5_0.5',
  'movesc_bid_2.5_0.7','movesc_ask_2.5_0.7','movesc_bid_3.5','movesc_ask_3.5','movesc_bid_3.5_0.7',
  'movesc_ask_3.5_0.7','movesc_bid_3.5_0.9','movesc_ask_3.5_0.9','movesc_bid_4.5','movesc_ask_4.5',
  'movesc_bid_4.5_0.5','movesc_ask_4.5_0.5','movesc_bid_4.5_0.7','movesc_ask_4.5_0.7','depth_bid7','depth_ask7'
  ]

l_side_drop_aux = ['big_c_aux','big_v_aux','aggpior_c_aux','aggpior_v_aux','loc_aggbig_c_m_aux',
  'loc_aggbig_v_m_aux','pagg_c_best_aux','pagg_c_best_0.5_aux','pagg_c_best_0.7_aux',
  'pagg_c_best_0.9_aux','pagg_v_best_aux','pagg_v_best_0.5_aux', 'pagg_v_best_0.7_aux',
  'pagg_v_best_0.9_aux'
  ]

l_ft_aux = [
  'vol_trd_aux', 'n_trd_aux','vol_big_aux','n_big_aux','vol_big_ratio_aux','big_c_aux','big_v_aux',
  'aggpior_c_aux','aggpior_v_aux','aggimb_aux','aggimb_big_aux','n_aggimb_aux','agg_net_m_aux',
  'aggbig_net_m_aux','loc_aggbig_c_m_aux','loc_aggbig_v_m_aux','loc_agg_net_m_aux','loc_aggbig_net_m_aux',
  'loc_agg_imb_m_aux','loc_aggbig_imb_m_aux','pagg_c_best_aux','pagg_c_best_0.5_aux','pagg_c_best_0.7_aux',
  'pagg_c_best_0.9_aux','pagg_v_best_aux','pagg_v_best_0.5_aux','pagg_v_best_0.7_aux','pagg_v_best_0.9_aux',
  'abagg_aux','n_p_aux','aggpior_aux_DIF','pagg_aux_DIF'
  ]



In [6]:
test_size = 0.2

if not EXPORT_X:
  df_X = tm_train.load_models('X_samples_'+s_prefix, os.path.join(path_files, 'TrainFiles'))  # search for path_files/s_regime.pkl

X = df_X.loc[df_X['model'] == s_regime].drop(columns='model')

if not EXPORT_Y:
  df_y = tm_train.load_models('y_' + y_config, os.path.join(path_files, 'TrainFiles'))
  y = df_y.loc[df_y['model'] == s_regime].drop(columns='model')
else:
  y = df_y.drop(columns='model')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, shuffle = False)

print('load_split: X, y data load and split complete!')



load_split: X, y data load and split complete!


In [7]:
# DEBUG: prop_features contaning np.NaN = (pd.isna(X_train).sum()>0).sort_values(ascending=False).head(50)
X_train = ft_eng.PropImputer(0.8, l_prop_08).transform(X_train)
X_train = ft_eng.PropImputer(1.2, l_prop_12).transform(X_train)
X_train = ft_eng.PropImputer(2, l_prop_2).transform(X_train)

median_inputer = MeanMedianImputer(variables=['PA_down',])
X_train = median_inputer.fit_transform(X_train)

nan_imputer = ArbitraryNumberImputer(0.0, variables=['ohlc_10','ohlc_50'])
X_train = nan_imputer.fit_transform(X_train)

X_train = ft_eng.DifAll().transform(X_train)
X_train = ft_eng.LogVolume(l_col_log).transform(X_train)

if s_regime[:2] == 'mw':
  # for now, removing all side columns
  l_cols_drop = l_side_drop + l_ft_aux + ['smart_price', 'sspread']
else:
  l_cols_drop = l_side_drop + l_side_drop_aux + ['s_run', 'n_p_aux', 'smart_price', 'sspread']

# TODO: INCREMENTAR L COLS DROP TAMBEM QUANDO LABEL NAO FOR DIST-TO-HIGH, RETIRANDO AS COLUNAS DE VARIACAO DE PRECO!

X_train.drop(columns=l_cols_drop, inplace=True)


In [8]:
X_test = ft_eng.PropImputer(0.8, l_prop_08).transform(X_test)
X_test = ft_eng.PropImputer(1.2, l_prop_12).transform(X_test)
X_test = ft_eng.PropImputer(2, l_prop_2).transform(X_test)

X_test = median_inputer.transform(X_test)

X_test = nan_imputer.transform(X_test)

X_test = ft_eng.DifAll().transform(X_test)
X_test = ft_eng.LogVolume(l_col_log).transform(X_test)

X_test.drop(columns=l_cols_drop, inplace=True)



In [9]:

# TODO: move function to ft_selection.py
def list_sub(lst1, lst2):
  final_list = list(set(lst1) - set(lst2))
  return final_list


def list_union(*l_lists):
  l_union = l_lists[0]
  if len(l_lists) > 1:
    for i in range(len(l_lists)-1):
      l_union = set(l_union) | set(l_lists[i+1])
  return list(l_union)

l_duplicate = ['loc_agg_net_m',]  # _2 ft comes from ft_eng.duplicate()

l_cap_1 = ['vewma_c_v', 'vewmag_dif', 'book_imb', 'book_imb_dif', ]

l_cap_5 = ['PA_up', 'PA_down', ]

l_cap_10 = ['n_big_aux', ]

l_cap_default = list_sub(X_train.columns.to_list(), list_union(l_cap_1, l_cap_5, l_cap_10))

l_bins_q2 = ['loc_agg_net_m_2',]  # _2 ft comes from ft_eng.duplicate()

l_bins_q4 = ['ohlc_10','ohlc_50',]

l_bins_q5 = ['loc_aggbig_net_m','chgfreq','vol_trd_aux','aggbig_net_m_aux','book_imb','rng_ewma_dif',
            'rng_ewma_dif_40','rng_ewma_dif_80','vewma_10','vewma_g_p_10','aggbig_net_10',]

l_bins_q6 = ['agg_net_d', 'aggbig_net_d', 'loc_agg_imb_m_aux',]

l_bins_q8 = ['rng_ewma','vewma_g_p','vewmag_dif','n_trd_aux','abagg_aux','escora_bid_2.5_0.8','escora_ask_2.5_0.8',
            'escora_bid_3.5_0.8','escora_ask_3.5_0.8','escora_bid_4.5_1.2','escora_ask_4.5_1.2','movesc_bid_2.5_0.9',
            'movesc_ask_2.5_0.9','movesc_bid_3.5_0.5','movesc_ask_3.5_0.5','movesc_bid_4.5_0.9','movesc_ask_4.5_0.9',
            'msg_imb','rng_smart_10','imp_DIF_10','imp_DIF_50','agg_net_80','imp_FCAST_40','aggpior_DIF',
            'book_imb_mean_dif_cp','msg_imb_mean_10','escora_4.5_0.8_DIF',]

l_bins_q10 = ['vol_big_ratio','aggimb','n_aggimb','agg_net_m','loc_aggbig_imb_m','abagg','n_p','vewma',
            'vewma_c_v', 'aggimb_aux','aggimb_big_aux','agg_net_m_aux','loc_aggbig_net_m_aux','smart_price_dif',
            'smart_price_50','rng_smart_50','agg_net_10','agg_net_40','loc_agg_net_10','int_DIF_10','abagg_10',
            'book_imb_mean_10','book_imb_mean_dif_lp','msg_imb_mean_40','msg_imb_mean_dif_lp','msg_imb_mean_dif_cp',
            'sspread_mean','movesc_2.5_0.7_DIF','msg_imb_mean_40_ABS','loc_agg_net_m',] 

d_bins_arbitrary = {
            'n_trd':  [-0.01, 70, 180, 280, 380, 5000000],
            'vol_trd':  [0, 7.237, 7.55, 10000],
            'aggimb_big':		[-1.1, -0.4, 0.4, 1.1],
            'aggbig_net_m':		[-10000, -5, +10000],
            'last_d_s':		[-0.01, 0.167, 0.280, 1],
            'loc_agg_net_d':		[-10000, -6.4, 5.2, 5.7, 10000],
            'loc_aggbig_net_d':		[-10000, -5.4, -3.8, 10000],
            'n_big_aux':	[-0.01, 3.1, 100],
            'vol_big_ratio_aux':		[-0.01, 0.275, 0.520, 1.01],
            'loc_agg_net_m_aux':		[-10000, -4.451, -3.592, -2.435, 3.607, 10000] ,
            'loc_aggbig_imb_m_aux':		[-1.1, -0.99, -0.391, -0.0118, 0.4, 0.99, 1.1],
            'aggpior_DIF_30':		[-7.625, -4.454, -4.111, -3.829, -3.26, -3.05, -3.04, -0.01, 0.01, 3.714, 7.6],
            'abs_DIF':		[-7.037, -4.796, -4.19, -0.1, 0.1, 2.398, 4.564, 7.19],
            'pagg_DIF':	[-1.1, -0.95, -0.6, -0.4, -0.001, 0.001, 0.4, 0.6, 0.95, 1.1],
            'book_imb_dif':	[-1000, -27, -14, -6.8, -2.5, 0.1, 4.29, 11.2, 22, 1000],
            'book_imb_mean_40': [-1000, -30, -20, -16, 1000],
            'aggpior_aux_DIF':		[-7.625, -4.331, -3.584, -3.045, -0.01, 0.01, 3.045, 7.6] ,
            'pagg_aux_DIF':		[-1.1, -0.95, -0.5, -0.001, 0.001, 0.5, 0.95, 1.1],
            'escora_3.5_1.2_DIF':		[-0.5, -0.171, -0.0482, 0, 0.0482, 0.0927, 0.5],
            'movesc_3.5_0.7_DIF':		[-17.5, -4.451, -1.391, -0.146, 0.0, 0.312, 8],
            'book_imb_mean_dif_cp_ABS':		[-0.01, 0.119, 500],
            'msg_imb_mean_dif_lp_ABS':		[-0.01, 0.281, 1.1],
            'loc_agg_imb_m':  [-1.01, -0.491, -0.366, -0.288, -0.223, -0.167, 1.01],
            'imp_FCAST_10': [-10000, -3.012, -1.674, -0.653, 10000],
}

l_bins_nulls = ['vol_big','n_big','PA_up','PA_down','vol_big_aux','n_aggimb_aux','n_p_aux','smart_price',
                'sspread','imp_DIF','int_DIF','int_DIF_50','abs_DIF_30','msg_imb_dif','depth_DIF',
                'depth_DIF_10','book_imb_mean_us_5','book_imb_mean_us_20','sspread_mean_us_5',
                'escora_2.5_2_DIF','movesc_4.5_0.7_DIF', ]


In [14]:
pipe_cap_outliers = Pipeline([
                ('cap1', Winsorizer(variables=l_cap_1, capping_method='quantiles', fold = 0.01, tail = 'both')),
                ('cap5', Winsorizer(variables=l_cap_5, capping_method='quantiles', fold = 0.05, tail = 'both')),
                ('cap10', Winsorizer(variables=l_cap_10, capping_method='quantiles', fold = 0.10, tail = 'both')),
                ('cap001', Winsorizer(variables=l_cap_default, capping_method='quantiles', fold = 0.001, tail = 'both')),
])

pipe_norm_scale = Pipeline([
                ('scaler', SklearnTransformerWrapper(transformer=RobustScaler({'quantile_range':(0.10, 0.90)}))),
                ('minmax', SklearnTransformerWrapper(transformer=MinMaxScaler())),
                # ('pca', PCA(n_components=30, svd_solver='auto')), 
                # ('isomap', Isomap(n_components=13, n_neighbors=50, n_jobs=-1)),   #  expensive
])

pipe_discrete = Pipeline([('drop', DropFeatures(l_bins_nulls)),
                        ('duplicate', ft_eng.Duplicate(l_duplicate)),
                        ('outliers_all', Winsorizer(capping_method='quantiles', fold = 0.001, tail = 'both')),
                        ('bins_manual', ArbitraryDiscretiser(d_bins_arbitrary)),
                        ('bins_q10', EqualFrequencyDiscretiser(return_object=True, q=10, variables=l_bins_q10)),
                        ('bins_q8', EqualFrequencyDiscretiser(return_object=True, q=8, variables=l_bins_q8)),
                        ('bins_q6', EqualFrequencyDiscretiser(return_object=True, q=6, variables=l_bins_q6)),
                        ('bins_q5', EqualFrequencyDiscretiser(return_object=True, q=5, variables=l_bins_q5)),
                        ('bins_q4', EqualFrequencyDiscretiser(return_object=True, q=4, variables=l_bins_q4)),
                        ('bins_q2', EqualFrequencyDiscretiser(return_object=True, q=2, variables=l_bins_q2)),
])

# X_train_pre = pipe_cap_outliers.fit_transform(X_train, y_train)
# X_test_pre = pipe_cap_outliers.transform(X_test)


C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass with_centering={'quantile_range': (0.1, 0.9)} as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [15]:

pipe_k_linear = Pipeline([('cap_outliers', pipe_cap_outliers),
                        ('norm_scale', pipe_norm_scale),]
)

pipe_clf = Pipeline([('pre_k_lin', pipe_k_linear), 
                    ('clf', SVC(probability=True, cache_size=1000, verbose=0
                                ,class_weight= 'balanced' ,C=1.26, gamma=0.0002))]
)

pipe_clf.fit(X_train, y_train) 
l_results = tm_train.report_results(X_train, X_test, y_train, y_test, fitted_model=pipe_clf, ready_probs=False)


C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
train_auc: 0.59, test_auc: 0.55 (overfit: 0.04)
train_f1:  0.60, test_f1:  0.56 (overfit: 0.04)
train_f*:  0.52, test_f*:  0.49 (overfit: 0.03)
--------
cm_train: 
[[2235 4796]
 [1010 4348]]
cm_test: 
[[ 566 1214]
 [ 323  995]]
threshold: 0.4


In [16]:
X_train.shape

(12389, 115)

In [17]:
pipe_svm_pca = Pipeline([('cap_outliers', pipe_cap_outliers),
                    ('norm_scale', pipe_norm_scale),
                    ('pca', PCA(n_components=30, svd_solver='auto')), 
                    ('svm', SVC(probability=True, cache_size=1000, verbose=0
                                ,class_weight= 'balanced' ,C=1.26, gamma=0.0002))
])

pipe_svm_pca.fit(X_train, y_train) 
l_results = tm_train.report_results(X_train, X_test, y_train, y_test, fitted_model=pipe_svm_pca, ready_probs=False)


C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
train_auc: 0.58, test_auc: 0.54 (overfit: 0.04)
train_f1:  0.60, test_f1:  0.55 (overfit: 0.04)
train_f*:  0.52, test_f*:  0.49 (overfit: 0.03)
--------
cm_train: 
[[2433 4598]
 [1139 4219]]
cm_test: 
[[ 617 1163]
 [ 370  948]]
threshold: 0.4


In [18]:
l_ft_alta = ['pagg_DIF','loc_agg_net_m_aux','aggpior_aux_DIF','loc_agg_imb_m','vol_big_ratio','aggimb',
             'n_aggimb','agg_net_m','loc_aggbig_imb_m','n_p','vewma','agg_net_m_aux','smart_price_dif',
             'smart_price_50','agg_net_10','msg_imb_mean_40_ABS','ohlc_10','ohlc_50','loc_aggbig_net_m',
             'vol_trd_aux','aggbig_net_m_aux','rng_ewma_dif_80','vewma_10','rng_ewma','vewma_g_p','vewmag_dif',
             'n_trd_aux','escora_bid_2.5_0.8','escora_ask_2.5_0.8','escora_bid_3.5_0.8','escora_ask_3.5_0.8'
             ,'escora_bid_4.5_1.2','escora_ask_4.5_1.2','movesc_bid_3.5_0.5','movesc_ask_3.5_0.5','msg_imb','rng_smart_10',
]

l_ft_media = ['last_d_s','aggimb_big','loc_aggbig_imb_m_aux','pagg_aux_DIF','movesc_3.5_0.7_DIF','abs_DIF',
              'book_imb_mean_dif_cp_ABS''book_imb_dif','abagg','vewma_c_v','aggimb_aux','loc_aggbig_net_m_aux',
              'rng_smart_50','agg_net_40','loc_agg_net_10','int_DIF_10','abagg_10','msg_imb_mean_dif_cp',
              'sspread_mean','movesc_2.5_0.7_DIF','loc_agg_net_m','book_imb','rng_ewma_dif','rng_ewma_dif_40',
              'vewma_g_p_10','aggbig_net_10','vol_trd','loc_agg_imb_m_aux','movesc_bid_2.5_0.9','movesc_ask_2.5_0.9',
              'movesc_bid_4.5_0.9','movesc_ask_4.5_0.9','imp_DIF_10','imp_FCAST_40','aggpior_DIF','book_imb_mean_dif_cp',
              'msg_imb_mean_10',
]



In [20]:
X_train.columns.to_list()

['vol_trd',
 'n_trd',
 'vol_big',
 'n_big',
 'vol_big_ratio',
 'aggimb',
 'aggimb_big',
 'n_aggimb',
 'agg_net_m',
 'aggbig_net_m',
 'loc_agg_net_m',
 'loc_aggbig_net_m',
 'loc_agg_imb_m',
 'loc_aggbig_imb_m',
 'abagg',
 'n_p',
 'agg_net_d',
 'aggbig_net_d',
 'chgfreq',
 'last_d_s',
 'PA_up',
 'PA_down',
 'loc_agg_net_d',
 'loc_aggbig_net_d',
 'vewma',
 'rng_ewma',
 'vewma_g_p',
 'vewma_c_v',
 'vewmag_dif',
 'vol_trd_aux',
 'n_trd_aux',
 'vol_big_aux',
 'n_big_aux',
 'vol_big_ratio_aux',
 'aggimb_aux',
 'aggimb_big_aux',
 'n_aggimb_aux',
 'agg_net_m_aux',
 'aggbig_net_m_aux',
 'loc_agg_net_m_aux',
 'loc_aggbig_net_m_aux',
 'loc_agg_imb_m_aux',
 'loc_aggbig_imb_m_aux',
 'abagg_aux',
 'book_imb',
 'escora_bid_2.5_0.8',
 'escora_ask_2.5_0.8',
 'escora_bid_3.5_0.8',
 'escora_ask_3.5_0.8',
 'escora_bid_4.5_1.2',
 'escora_ask_4.5_1.2',
 'movesc_bid_2.5_0.9',
 'movesc_ask_2.5_0.9',
 'movesc_bid_3.5_0.5',
 'movesc_ask_3.5_0.5',
 'movesc_bid_4.5_0.9',
 'movesc_ask_4.5_0.9',
 'msg_imb',
 'smart_

In [21]:
l_selection_drop

['movesc_ask_4.5_0.9',
 'msg_imb_mean_40',
 'vol_big_aux',
 'loc_agg_imb_m_aux',
 'agg_net_80',
 'vol_big',
 'imp_FCAST_40',
 'aggpior_DIF',
 'last_d_s',
 'loc_agg_net_10',
 'loc_agg_net_d',
 'n_big',
 'pagg_aux_DIF',
 'rng_ewma_dif',
 'loc_aggbig_net_d',
 'aggbig_net_10',
 'int_DIF_10',
 'vewma_c_v',
 'aggpior_DIF_30',
 'aggbig_net_d',
 'abs_DIF_30',
 'abagg_10',
 'msg_imb_mean_10',
 'msg_imb_dif',
 'movesc_bid_2.5_0.9',
 'loc_agg_net_m',
 'loc_aggbig_net_m_aux',
 'book_imb_mean_dif_cp_ABS',
 'book_imb_mean_dif_lp',
 'movesc_ask_2.5_0.9',
 'sspread_mean',
 'book_imb_dif',
 'aggimb_big_aux',
 'book_imb_mean_10',
 'PA_down',
 'n_trd',
 'movesc_bid_4.5_0.9',
 'movesc_2.5_0.7_DIF',
 'book_imb_mean_us_20',
 'rng_smart_50',
 'abs_DIF',
 'abagg',
 'book_imb',
 'escora_2.5_2_DIF',
 'aggimb_big',
 'agg_net_d',
 'n_aggimb_aux',
 'agg_net_40',
 'sspread_mean_us_5',
 'vol_trd',
 'depth_DIF',
 'vol_big_ratio_aux',
 'depth_DIF_10',
 'book_imb_mean_dif_cp',
 'movesc_4.5_0.7_DIF',
 'abagg_aux',
 'esc

In [22]:
l_selection_drop = list_sub(X_train.columns.to_list(), l_ft_alta)
pipe_svm_ft_alta = Pipeline([('cap_outliers', pipe_cap_outliers),
                            ('norm_scale', pipe_norm_scale),
                            ('keep_alta', DropFeatures(l_selection_drop)),
                            ('svm', SVC(probability=True, cache_size=1000, verbose=0
                                        ,class_weight= 'balanced' ,C=1.26, gamma=0.0002))
])

pipe_svm_ft_alta.fit(X_train, y_train) 
l_results = tm_train.report_results(X_train, X_test, y_train, y_test, fitted_model=pipe_svm_ft_alta, ready_probs=False)


C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
train_auc: 0.59, test_auc: 0.54 (overfit: 0.04)
train_f1:  0.61, test_f1:  0.60 (overfit: 0.01)
train_f*:  0.49, test_f*:  0.48 (overfit: 0.01)
--------
cm_train: 
[[  92 6939]
 [  13 5345]]
cm_test: 
[[  18 1762]
 [   4 1314]]
threshold: 0.4


In [23]:
l_results = tm_train.report_results(X_train, X_test, y_train, y_test, fitted_model=pipe_svm_ft_alta, ready_probs=False, th=0.5)


train_auc: 0.59, test_auc: 0.54 (overfit: 0.04)
train_f1:  0.00, test_f1:  0.00 (overfit: 0.00)
train_f*:  0.00, test_f*:  0.00 (overfit: 0.00)
--------
cm_train: 
[[7030    1]
 [5357    1]]
cm_test: 
[[1780    0]
 [1318    0]]
threshold: 0.5


In [24]:
l_results = tm_train.report_results(X_train, X_test, y_train, y_test, fitted_model=pipe_svm_ft_alta, ready_probs=False, th=0.45)


train_auc: 0.59, test_auc: 0.54 (overfit: 0.04)
train_f1:  0.55, test_f1:  0.50 (overfit: 0.05)
train_f*:  0.51, test_f*:  0.47 (overfit: 0.04)
--------
cm_train: 
[[3331 3700]
 [1912 3446]]
cm_test: 
[[880 900]
 [581 737]]
threshold: 0.5


In [25]:
1/30

0.03333333333333333

In [26]:
pipe_svm_ft_alta = Pipeline([('cap_outliers', pipe_cap_outliers),
                            ('norm_scale', pipe_norm_scale),
                            ('keep_alta', DropFeatures(l_selection_drop)),
                            ('svm', SVC(probability=True, cache_size=1000, verbose=0,
                                        class_weight= 'balanced' ,C=1.26, gamma=0.01))
])

pipe_svm_ft_alta.fit(X_train, y_train) 
l_results = tm_train.report_results(X_train, X_test, y_train, y_test, fitted_model=pipe_svm_ft_alta, ready_probs=False)


C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
train_auc: 0.61, test_auc: 0.57 (overfit: 0.04)
train_f1:  0.60, test_f1:  0.55 (overfit: 0.05)
train_f*:  0.53, test_f*:  0.50 (overfit: 0.03)
--------
cm_train: 
[[2638 4393]
 [1180 4178]]
cm_test: 
[[ 733 1047]
 [ 414  904]]
threshold: 0.4


In [28]:
pipe_svm_pca = Pipeline([('cap_outliers', pipe_cap_outliers),
                    ('norm_scale', pipe_norm_scale),
                    ('pca', PCA(n_components=20, svd_solver='auto')), 
                    ('svm', SVC(probability=True, cache_size=1000, verbose=0
                                ,class_weight= 'balanced' ,C=1.26, gamma=0.01))
])



In [29]:
pipe_svm_pca.fit(X_train, y_train) 
l_results = tm_train.report_results(X_train, X_test, y_train, y_test, fitted_model=pipe_svm_pca, ready_probs=False, th=0)


C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
train_auc: 0.61, test_auc: 0.56 (overfit: 0.05)
train_f1:  0.60, test_f1:  0.55 (overfit: 0.05)
train_f*:  0.52, test_f*:  0.49 (overfit: 0.03)
--------
cm_train: 
[[2660 4371]
 [1229 4129]]
cm_test: 
[[ 779 1001]
 [ 446  872]]
threshold: 0.4


In [31]:
pipe_discrete = Pipeline([('drop', DropFeatures(l_bins_nulls)),
                        ('duplicate', ft_eng.Duplicate(l_duplicate)),
                        ('outliers_all', Winsorizer(capping_method='quantiles', fold = 0.001, tail = 'both')),
                        ('bins_manual', ArbitraryDiscretiser(d_bins_arbitrary)),
                        ('bins_q10', EqualFrequencyDiscretiser(return_object=False, q=10, variables=l_bins_q10)),
                        ('bins_q8', EqualFrequencyDiscretiser(return_object=False, q=8, variables=l_bins_q8)),
                        ('bins_q6', EqualFrequencyDiscretiser(return_object=False, q=6, variables=l_bins_q6)),
                        ('bins_q5', EqualFrequencyDiscretiser(return_object=False, q=5, variables=l_bins_q5)),
                        ('bins_q4', EqualFrequencyDiscretiser(return_object=False, q=4, variables=l_bins_q4)),
                        ('bins_q2', EqualFrequencyDiscretiser(return_object=False, q=2, variables=l_bins_q2)),
])

pipe_mean_encoding = Pipeline([('drop', DropFeatures(l_bins_nulls)),
                        ('duplicate', ft_eng.Duplicate(l_duplicate)),
                        ('outliers_all', Winsorizer(capping_method='quantiles', fold = 0.001, tail = 'both')),
                        ('bins_manual', ArbitraryDiscretiser(d_bins_arbitrary)),
                        ('bins_q10', EqualFrequencyDiscretiser(return_object=True, q=10, variables=l_bins_q10)),
                        ('bins_q8', EqualFrequencyDiscretiser(return_object=True, q=8, variables=l_bins_q8)),
                        ('bins_q6', EqualFrequencyDiscretiser(return_object=True, q=6, variables=l_bins_q6)),
                        ('bins_q5', EqualFrequencyDiscretiser(return_object=True, q=5, variables=l_bins_q5)),
                        ('bins_q4', EqualFrequencyDiscretiser(return_object=True, q=4, variables=l_bins_q4)),
                        ('bins_q2', EqualFrequencyDiscretiser(return_object=True, q=2, variables=l_bins_q2)),
                        ('mean_enc', MeanEncoder())
])



In [33]:
l_bins_nulls = ['vol_big','n_big','PA_up','PA_down','vol_big_aux','n_aggimb_aux','imp_DIF','int_DIF',
                'int_DIF_50','abs_DIF_30','msg_imb_dif','depth_DIF',
                'depth_DIF_10','book_imb_mean_us_5','book_imb_mean_us_20','sspread_mean_us_5',
                'escora_2.5_2_DIF','movesc_4.5_0.7_DIF', 
                # TODO: 's_run', 'n_p_aux' treat for MW
]



In [34]:
pipe_discrete = Pipeline([('drop', DropFeatures(l_bins_nulls)),
                        ('duplicate', ft_eng.Duplicate(l_duplicate)),
                        ('outliers_all', Winsorizer(capping_method='quantiles', fold = 0.001, tail = 'both')),
                        ('bins_manual', ArbitraryDiscretiser(d_bins_arbitrary)),
                        ('bins_q10', EqualFrequencyDiscretiser(return_object=False, q=10, variables=l_bins_q10)),
                        ('bins_q8', EqualFrequencyDiscretiser(return_object=False, q=8, variables=l_bins_q8)),
                        ('bins_q6', EqualFrequencyDiscretiser(return_object=False, q=6, variables=l_bins_q6)),
                        ('bins_q5', EqualFrequencyDiscretiser(return_object=False, q=5, variables=l_bins_q5)),
                        ('bins_q4', EqualFrequencyDiscretiser(return_object=False, q=4, variables=l_bins_q4)),
                        ('bins_q2', EqualFrequencyDiscretiser(return_object=False, q=2, variables=l_bins_q2)),
])

pipe_mean_encoding = Pipeline([('drop', DropFeatures(l_bins_nulls)),
                        ('duplicate', ft_eng.Duplicate(l_duplicate)),
                        ('outliers_all', Winsorizer(capping_method='quantiles', fold = 0.001, tail = 'both')),
                        ('bins_manual', ArbitraryDiscretiser(d_bins_arbitrary)),
                        ('bins_q10', EqualFrequencyDiscretiser(return_object=True, q=10, variables=l_bins_q10)),
                        ('bins_q8', EqualFrequencyDiscretiser(return_object=True, q=8, variables=l_bins_q8)),
                        ('bins_q6', EqualFrequencyDiscretiser(return_object=True, q=6, variables=l_bins_q6)),
                        ('bins_q5', EqualFrequencyDiscretiser(return_object=True, q=5, variables=l_bins_q5)),
                        ('bins_q4', EqualFrequencyDiscretiser(return_object=True, q=4, variables=l_bins_q4)),
                        ('bins_q2', EqualFrequencyDiscretiser(return_object=True, q=2, variables=l_bins_q2)),
                        ('mean_enc', MeanEncoder())
])

